# Diode ring mixer
Minicircuits : [ADE-1+, Level 7, SMT Double Balanced Mixer, RF/LO Freq 0.5 - 500 MHz](https://www.minicircuits.com/WebStore/dashboard.html?model=ADE-1%2B)
[#260: RF Diode Mixer LO Drive Level & Conversion Loss | 1dB Compression | Distortion](https://www.youtube.com/watch?v=GvadQpkZ8l0)

## Driving the LO port
* [Reducing IMD in high-level mixers](https://www.robkalmeijer.nl/techniek/electronica/radiotechniek/hambladen/qex/2001/05_06/page45/index.html)
* LO drive should be as symmetrical as possible (50% duty cycle). A square wave is ideal.
* Add a BPF for the 2nd harmonic of the LO signal to the RF port. A square wave has no 2nd harmonic content anyway.  This will help to reduce the IMD products.


In [31]:
import math
# LO side injection, so LO runs from 88-10.7MHz to 108-10.7MHz
# LO center frequency is 98MHz - 10.7MHz = 87.3MHz
f_0 = 87.3e6  # LO center frequency
ω = 2 * math.pi * f_0
BW = 20e6  # Bandwidth
Q = f_0 / BW # Q factor of the filter

Rin = 50 # 50 Ohm, input impedance (LO)
Rout = 50 # 50 Ohm, output impedance (LO-port of the mixer)
R_L = Rin * Rout / (Rin + Rout) # l

X_0 = R_L / Q # reactance at center frequency
L = X_0 / ω # inductance at center frequency
print("L = ", L * 1e6, "uH")
C = 1 / (ω**2 * L) # capacitance at center frequency
print("C = ", C * 1e12, "pF")

L =  0.010441470085241171 uH
C =  318.3098861837907 pF


A low-pass filter messes up the switching waveform.  A source termination resistor will be used instead to provide a nice clean waveform without over/undershoot.


## Termination
EMRFD : S11 should be -10dB. Q should be between 1 and 10.

### Bandpass/bandstop Diplexer
Highpass/lowpass diplexers are not suitable for this application.  So J.J. Carr, RF components and circuits, fig 5.32 can not be used.

* J.J. Carr, RF components and circuits, p. 116
* J.J.Carr Secrets of RF circuit design, §20 Using the DBM (p. 347) : diplexer, pre-and post-mixer amplifiers

* 3dB pad (reduces NF by 3dB): see Mini-circuits FAQ
* EMRFD : 3dB pads applied to all three ports of the mixer.
* broadband amplifier 
* terminate all three ports
* [Reducing IMD in high-level mixers](https://www.robkalmeijer.nl/techniek/electronica/radiotechniek/hambladen/qex/2001/05_06/page45/index.html)

https://www.eevblog.com/forum/rf-microwave/double-balanced-mixers-various-questions-probems-with-my-first-attempt/
* Terminate the IF-port into a diplexer. The IF port needs a good broadband 50ohm match over the widest bandwidth you can get.
* Terminate the RF-port into a well matched 50ohm bandpass filter, or 50ohm source impedance.
Don't worry about the LO source impedance, it is far less important. Just make sure you have enough LO to drive the diodes, and a square wave with good sharp, symmetrical edges will give approx. 10 to 20dB of improvement in the intermodulation performance.
Make your transformers identical. They aren't just transformers, they're transmission lines. They are doing an impedance transformation, and they need to be as tightly matched as you can possibly make them. Trifiliar windings are a must.


In [32]:
import math
# Sayre §6.4 : this only provides matching from the LO-port.  The output of the filter must be 50ohm.  There's no backward matching (S22 is 0dB)
# Hayward's IRFD, fig 4.26b proposes double ended termination to solve this.

BW = 200e3  # Bandwidth
f_0 = 10.7e6  # IF frequency
ω = 2 * math.pi * f_0
# Q = f_0 / BW # Q factor of the filter
Q = 10 # Q factor of the filter

# The formulas given here are lumped filter image parameters.  These are explained well in Sayre §6.2.3

print("Q = ", Q)
R_L = 50 # load resistance (IF port of the mixer)
L_1 = R_L / (ω * Q) # inductance at center frequency
print("L_1 = ", L_1 * 1e6, "uH")
C_1 = 1 / (ω**2 * L_1) # capacitance at center frequency
print("C_1 = ", C_1 * 1e12, "pF")
L_2 = Q * R_L / ω # inductance at center frequency
print("L_2 = ", L_2 * 1e6, "uH")
C_2 = 1 / (ω**2 * L_2) # capacitance at center frequency
print("C_2 = ", C_2 * 1e12, "pF")

Q =  10
L_1 =  0.07437146873453052 uH
C_1 =  2974.858749381222 pF
L_2 =  7.437146873453053 uH
C_2 =  29.74858749381222 pF


In [33]:
# References: Bandpass / bandstop diplexer
# * EMRFD, p.3.38
# * J.J.Carr : "RF Componenents and Circuits" fig. 5.31

# S11 equals S22, which are both very good.

def calc_filter(f_0, Q):
    ω = 2 * math.pi * f_0
    R = 50 # load resistance
    L = 50 * Q / ω # inductance at center frequency
    print("L = ", L * 1e6, "uH")
    C = 1 / (ω**2 * L) # capacitance at center frequency
    print("C = ", C * 1e12, "pF")
    Cp = L / R**2
    print("Cp = ", Cp * 1e12, "pF")
    Lp = 1 / (ω**2 * Cp)
    print("Lp = ", Lp * 1e6, "uH")

calc_filter(10.7e6, 10) # IF filter port
BW = 20e6  # Bandwidth
calc_filter(98e6, 98e6 / BW) # RF filter port

L =  7.437146873453053 uH
C =  29.74858749381222 pF
Cp =  2974.858749381221 pF
Lp =  0.07437146873453054 uH
L =  0.39788735772973843 uH
C =  6.628694006326335 pF
Cp =  159.15494309189538 pF
Lp =  0.01657173501581584 uH


In [34]:
# J.J.Carr : "Secrets of RF Circuit Design" p. 195, fig. 20-3
# Similar to J.J.Carr : "RF Componenents and Circuits" fig. 5.31, but with a different component values.

# The component values are wrong.

f_0 = 10.7e6  # IF frequency
ω = 2 * math.pi * f_0
L = 50 / ω # inductance at center frequency
print("L = ", L * 1e6, "uH")
C = 1 / (ω * 50) # capacitance at center frequency
print("C = ", C * 1e12, "pF")

L =  0.7437146873453053 uH
C =  297.48587493812215 pF


# Diplexer actual design
## IF port diplexer

In [ ]:
Q = 10 # Q factor of the filter
f_0 = 10.7e6  # IF frequency
ω = 2 * math.pi * f_0
R = 50 # load resistance
Lp = R / (ω * Q) # inductance at center frequency
print("Lp = ", L * 1e9, "nH")

# Select Lp for highest Q at 10.7MHz → L = 744912168G : 68nH
Lp = 68e-9 # inductance at center frequency

# Calculate Cp for 10.7MHz
Cp = 1 / (ω**2 * Lp) # capacitance at center frequency
print("Cp = ", Cp * 1e9, "nF")

# Select L for highest Q at 10.7MHz → L = 
L = 50 * Q / ω # inductance at center frequency
print("L = ", L * 1e9, "nH")

# Calculate C for 10.7MHz
C = 1 / (ω**2 * L) # capacitance at center frequency
print("C = ", C * 1e12, "pF")


Lp =  74.37146873453052 nH
Cp =  3.2535972716066195 nF
L =  7437.146873453053 nH
C =  29.74858749381222 pF


## RF port diplexer

In [ ]:
BW = 20e6
f_0 = 98e6  # RF frequency, middle of the band
Q = f_0 / BW # Q factor of the filter
ω = 2 * math.pi * f_0
R = 50 # load resistance
Lp = R / (ω * Q) # inductance at center frequency
print("Lp = ", L * 1e9, "nH")

# # Select Lp for highest Q at 98MHz
Lp = 17.5e-9 # L = 744914117 : 17.5nH

# Calculate Cp for 98MHz
Cp = 1 / (ω**2 * Lp) # capacitance at center frequency
print("Cp = ", Cp * 1e12, "pF")

# Select L for highest Q at 98MHz
L = 50 * Q / ω # inductance at center frequency
print("L = ", L * 1e9, "nH")
L = 390e-9 # L = 744762239A : 390nH

# Calculate C for 98MHz
C = 1 / (ω**2 * L) # capacitance at center frequency
print("C = ", C * 1e12, "pF")


Lp =  7437.146873453053 nH
Cp =  150.71277390720797 pF
L =  397.8873577297384 nH
C =  6.762752675323434 pF


# Implementation of high frequency 7.4µH inductor
[Toroids - Some practical considerations](https://www.robkalmeijer.nl/techniek/electronica/radiotechniek/hambladen/qst/1988/12/page33/index.html)
## Iron powdered core
[Amidon iron powder toroids calculator](https://coil32.net/online-calculators/amidon-iron-powder-cores-calculator.html)
J.J. Carr, RF components and circuits, p.243
* Material : type 6 : 10MHz → 50MHz
* Size : T50 → A_L = 43µH/100t²
* Turns = 41
* Wire size : AWG26 or smaller
* [T37-6](https://www.amidoncorp.com/t37-6/) : 50 turns

In [37]:
# Iron powdered inductor
A_L = 43 # µH/(100t)²
L = 7.4 # µH
N = 100 * math.sqrt(L / A_L)
print("Iron-powdered N = ", N)

Iron-powdered N =  41.4840961400648



## Ferrite core
[Amidon ferrite toroids calculator](https://coil32.net/online-calculators/amidon-ferrite-torroid-calculator.html)
J.J. Carr, RF components and circuits, p.243
* Material : [type 61](https://fair-rite.com/61-material-data-sheet/)
  * [FT-37-61](https://www.amidoncorp.com/ft-37-61/)
  * [Fair-rite 5961000201](https://fair-rite.com/product/toroids-5961000201/)
* Material : [type 67](https://fair-rite.com/67-material-data-sheet/) : 10MHz → 80MHz
  * [FT-37-67](https://www.amidoncorp.com/ft-37-67/) → A_L = 18nH/N², €1
  * [Fair-rite 5967000201](https://fair-rite.com/product/toroids-5967000201/)

In [38]:
# Ferrite core inductor
A_L = 55 # nH/N²
L = 7.4e3 # nH
N = math.sqrt(L / A_L)
print("Ferrite core N = ", N)

Ferrite core N =  11.599373023808422



# Gilbert cell mixer
ONsemi : MC1496 / LM1496
* requires high supply voltage (12V and or -8V)
* LT5560 with Balun : WBC1-1TL 1:1

https://radioaficionsdr.es.tl/Mezcladores-de-RF.htm
https://www.qsl.net/va3iul/RF%20Mixers/RF_Mixers.pdf : mixer termination in last pages

# Active Dual gate MOSFET mixer
Dual gate MOSFETs can be replaced by 2N-channel JFETs
JFETs on Digikey : https://www.digikey.be/en/products/filter/transistors/fets-mosfets/rf-fets-mosfets/285?s=N4IgjCBcoKwOwAYqgMZQGYEMA2BnApgDQgD2UA2iHGAJwBsALHCALrEAOALlCAMqcAnAJYA7AOYgAvsQC0AJmQg0kQQFcipCiBghiNVtJAyAzIuVqNZSJTBIWkh0A
* [Mixer testing of FM-receiver: BF990A](https://www.rfcandy.biz/communication/mxtest.html)
* [understanding-a-mosfet-mixer](https://hackaday.com/2018/06/11/understanding-a-mosfet-mixer/)

# Passive JFET mixer
[pe4140ds](https://www.psemi.com/pdf/datasheets/pe4140ds.pdf)


# References
Jessop p.24 : mixer

https://www.qsl.net/va3iul/Homebrew_RF_Circuit_Design_Ideas/Homebrew_RF_Circuit_Design_Ideas.htm#Receivers

